# Pandas Challenge: Revenue

We'll use fake sales data to cover some Pandas functions, including merge, groupby, query, and diff. 

In [ ]:
# imports
import random

import numpy as np
import pandas as pd

from datetime import datetime

# Data

### A sales person is assigned to an ad in given date range

In [44]:
date_end = pd.date_range(pd.to_datetime("2021-03-01"), periods=100).tolist()

In [45]:
date_start = pd.date_range(pd.to_datetime("2020-01-01"), periods=100).tolist()

In [46]:
ids = range(100)

In [47]:
sales = range(10000, 10100)

In [48]:
ads = pd.DataFrame({
    "ad": range(100),
    "date_start": date_start,
    "date_end": date_end,
    "sales": sales,
})

In [49]:
ads.head()

,ad,date_start,date_end,sales
0,0,2020-01-01,2021-03-01,10000
1,1,2020-01-02,2021-03-02,10001
2,2,2020-01-03,2021-03-03,10002
3,3,2020-01-04,2021-03-04,10003
4,4,2020-01-05,2021-03-05,10004


### Revenue is generated by an ad on a given date

In [78]:
clicks = ads.sample(n=20).reset_index(drop=True)

In [79]:
clicks.drop(columns=["date_end", "sales"], inplace=True)

In [80]:
clicks.rename(columns={"date_start": "date_revenue"}, inplace=True)

In [81]:
clicks["revenue"] = sampled["ad"] * 1000 * random.random()
clicks["revenue"] = sampled["revenue"].astype(int)

In [86]:
clicks["date_revenue"] = clicks["date_revenue"] + pd.DateOffset(1)

In [87]:
clicks.head()

,ad,date_revenue,revenue
0,88,2020-03-30,1442
1,43,2020-02-14,1590
2,37,2020-02-08,409
3,73,2020-03-15,32
4,13,2020-01-15,688


# Problems

## What is the sum of revenue generated by sales? And the average?

In [250]:
click_ads = ads.merge(clicks, on="ad")

In [252]:
click_ads.head()

,ad,date_start,date_end,sales,date_revenue,revenue
0,13,2020-01-14,2021-03-14,10013,2020-01-15,688
1,17,2020-01-18,2021-03-18,10017,2020-01-19,295
2,27,2020-01-28,2021-03-28,10027,2020-01-29,311
3,28,2020-01-29,2021-03-29,10028,2020-01-30,1180
4,33,2020-02-03,2021-04-03,10033,2020-02-04,1524


In [266]:
sales_revenue = click_ads.query("(date_revenue <= date_end) & (date_revenue >= date_start)")
sales_revenue.agg(['mean', 'sum'])["revenue"]

mean      874.05
sum     17481.00
Name: revenue, dtype: float64

## What is the longest consecutive period of revenue per ad?

In [195]:
# add new fake data to have consecutive runs per ad
def make_clicks(data, n=1):
    df = data.copy()
    df["date_revenue"] = df["date_revenue"] + pd.DateOffset(n)
    return df

clicks_1 = clicks.copy()
more_clicks = clicks.copy()

In [196]:
for n in range(10, 14):
    cl = make_clicks(clicks_1, n)
    more_clicks = more_clicks.append(cl)

In [197]:
for n in range(100, 110):
    cl = make_clicks(clicks_1, n)
    more_clicks = more_clicks.append(cl)

In [198]:
df = more_clicks.sort_values(["ad", "date_revenue"])

In [243]:
df["diffed"] = df["date_revenue"].diff().dt.days == 1.0

In [244]:
df.head()

,ad,date_revenue,revenue,diffed,groups
4,13,2020-01-15,688,False,1
4,13,2020-01-25,688,False,2
4,13,2020-01-26,688,True,2
4,13,2020-01-27,688,True,2
4,13,2020-01-28,688,True,2


In [245]:
# consecutive runs
df["groups"] = (~df["diffed"]).cumsum()

In [241]:
# filter out first entry of run
runs = df[df["diffed"]].groupby(["ad", "groups"]).sum()["diffed"]

In [242]:
runs.max(level=0)

ad
13    9
17    9
27    9
28    9
33    9
37    9
43    9
45    9
50    9
52    9
60    9
66    9
71    9
73    9
76    9
82    9
84    9
88    9
92    9
96    9
Name: diffed, dtype: int64